## Assigment-01

### 基于模式匹配的对话机器人


### 1. Pattern Match
Pattern: (我想要A)     
Response:（如果你有A, 对你意味着什么呢？）     

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成 "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [10]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def pat_match(pattern, saying):
    "Define a simple proceture that checks whether the saying matches with the pattern"
    
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] !=saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

# This isalpha() returns True if all characters in the strings are alphabets and 
# False if at least one character is not alphabet.

In [2]:
pat_match('I want ?X'.split(), 'I want holiday'.split())

True

In [3]:
pat_match('I dreamed about ?X'.split(), 'I have dreamed a dog'.split())

False

In [4]:
def pat_match(pattern, saying):
    "Revise the previous procedure to return the matched variable <A>."
    
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])


In [5]:
pat_match('I want ?X'.split(), 'I want holiday'.split())

('?X', 'holiday')

In [6]:
pat_match('?X equals ?X'.split(), '2+2 equals 4'.split())
# Note that when we have two pairs matched variables, the procedure only returns the first one

('?X', '2+2')

In [7]:
def pat_match(pattern, saying):
    "Revise the previous procedure so that we can identify all matched pairs."
    
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])
        

In [8]:
pat_match('?X is greater than ?Y'.split(), '3 is greater than 2'.split())

[('?X', '3'), ('?Y', '2')]

In [111]:
# Now we want to do two more things --- a) define a procedure that translates the matched relations
# into a dictionary; and b) define a procedure that can substitute variables given a dictionary
def pat_to_dic (patterns):
    return{k:v for k, v in patterns}

def substitute(pattern, pat_dic):
    if not pattern: return []
    
    return [pat_dic.get(pattern[0], pattern[0])] + substitute(pattern[1:], pat_dic)


In [10]:
got_patterns = pat_match('I want ?X'.split(), 'I want iPhone'.split())

In [11]:
got_patterns

[('?X', 'iPhone')]

In [13]:
substitute("What do you do if you got a ?X".split(), pat_to_dic(got_patterns))

['What', 'do', 'you', 'do', 'if', 'you', 'got', 'a', 'iPhone']

In [14]:
' '.join(substitute("What do you do if you got a ?X".split(), pat_to_dic(got_patterns)))
# join the sentence together

'What do you do if you got a iPhone'

In [15]:
# try another example
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [17]:
' '.join(substitute('Why does ?P need ?X'.split(), pat_to_dic(john_pat)))

'Why does John need vacation'

In [77]:
# Now we can define a pattern dictionary
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

[key.split() for key in defined_patterns.keys()]
defined_patterns.get("I need ?X")

['Image you will get ?X soon', 'Why do you need ?X ?']

In [91]:
import random
def get_response(saying, pat_dic):
    "Define a procedure that returns a patterned response based the dictionary supplied"
    
    if not saying or not pat_dic: return []
    
    response_list = []
    
    for key, value in pat_dic.items():
        got_patterns = pat_match(key.split(), saying.split())
        if got_patterns:
            pat_response = pat_dic.get(key)
            for r in pat_response:
                response_list.append(' '.join(substitute(r.split(), pat_to_dic(got_patterns))))
        else: continue
    
    if response_list: 
        return random.choice(response_list)
    else: 
        return "Sorry, I don't know how to answer."


In [95]:
get_response('My mom told me something', defined_patterns)

'Talk about more about your mom'

In [96]:
get_response('I need vacation', defined_patterns)

'Why do you need vacation ?'

### 2. Segment Match
我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable.


In [9]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [26]:
is_pattern_segment('?*PYAMC')

True

In [43]:
from collections import defaultdict

fail = [True, None]


def pat_match_with_seg(pattern, saying):
    "Revise the previous pat_match in order to match ?*x variable with a segment of texts."
    
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):    # when we have a ?X variable
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):   # when we have a ?*X variable
        if segment_match(pattern, saying) != fail:
            match, index = segment_match(pattern, saying)
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
        else: 
            return segment_match(pattern, saying) # when the segment_match returns fail
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    

def segment_match(pattern, saying):
    "Find the group of words in saying that matches with the ?*x variable in a given pattern."
    
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')  # change the ?* prefix into a single ?
    
    if not rest: return (seg_pat, saying), len(saying) 
    
    for i, token in enumerate(saying):
        if rest[0] == token: 
            return (seg_pat, saying[:i]), i    # note i = len(saying) :)
    
    return fail # to prevent ?*X matches the whole of saying

In [44]:
segment_match('?*P is very good'.split(), "My dog along with my cat is very good".split())

(('?P', ['My', 'dog', 'along', 'with', 'my', 'cat']), 6)

In [45]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [46]:
pat_match_with_seg('?*P are good'.split(), "My cat is very cute".split())

[True, None]

In [47]:
# Give some example patterns in order to construct responses
response_pair = {
    'I need ?X': ['Why do you need ?X'],
    "I don't like my ?X": ["What bad things did ?X do for you?"]
}

In [48]:
pat_match_with_seg('I need ?*X'.split(), "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [88]:
got_pattern = pat_match_with_seg('I need ?*X'.split(), "I need an iPhone".split())
substitute("Why do you need ?X".split(), pat_to_dic(got_pattern))

['Why', 'do', 'you', 'need', 'an iPhone']

In [126]:
# Revise the pat_to_dic procedure to return a joined phrase for the substituted values of ?X
#  instead of broken words!

def pat_to_dic(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k,v in patterns} 
# join v if v is a list

In [89]:
substitute("Why do you need ?X".split(), pat_to_dic(got_pattern))
' '.join(substitute("Why do you need ?X".split(), pat_to_dic(got_pattern)))

'Why do you need an iPhone'

In [52]:
# If we have a pattern like this one:

("?*X hello ?*Y", "Hi, how do you do")
pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())

[('?X', ['I', 'am', 'mike,'])]

In [136]:
got_pattern = pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())
substitute("Hi, how do you do?".split(), pat_to_dic(got_pattern))

['Hi,', 'how', 'do', 'you', 'do?']

In [60]:
rules = {
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    "?*X hello ?*Y": ["Hi, how do you do?"]
}

#### 问题 1
编写一个程序, get_response(saying, response_rules)输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [127]:
# Now we want to define a procedure, get_response, that does the following things:
#  -- takes two inputs, a "saying", and a dictionary that specify a set of response rules，
#     where each element in the dictionary follows the format 
#     {<pattern>: [<response1>,<response2>,...]}
#  -- it finds a matched pattern within the dictionary supplied for the "saying" (pat_match_with_seg)
#  -- it finds the key words within the "saying" used as substitute and gives a patterned response (substitute, pat_to_dic)
#  -- to make the response interesting, it randomly chooses a patterned response for a list of possible responses defined by the rules

import random
def get_response(saying, pat_dic):
    if not saying or not pat_dic: return []
    
    response_list = []
    
    for key, value in pat_dic.items():
        got_patterns = pat_match_with_seg(key.split(), saying.split())
        if got_patterns != fail:
            pat_response = pat_dic[key]
            for r in pat_response:
                response_list.append(' '.join(substitute(r.split(), pat_to_dic(got_patterns))))
        else: continue
    
    if response_list: 
        return random.choice(response_list)
    else: 
        return "Sorry, I don't know how to answer."


In [131]:
pat_match_with_seg("?*X hello ?*Y".split(), "I was a scientist".split())

[True, None]

In [138]:
segment_match("?*X hello ?*Y".split(), "I am mike, hello".split())

(('?X', ['I', 'am', 'mike,']), 3)

In [121]:
pat_match_with_seg("I was ?*X".split(), "I was a scientist".split())

[('?X', ['a', 'scientist'])]

In [128]:
get_response("I was a historian", rules)

'Were you really a historian ?'

In [129]:
get_response("Hi, my name is John", rules)

"Sorry, I don't know how to answer."

In [140]:
get_response("I am Mike, hello", rules)

'Hi, how do you do?'

#### 问题 2
将以上程序改写为能够支持中文输入的模式（提示：jieba分词）

In [141]:
import jieba
import random
from collections import defaultdict

fail = [True, None]


def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

def pat_to_dic(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k,v in patterns} 

def substitute(pattern, pat_dic):
    if not pattern: return []
    return [pat_dic.get(pattern[0], pattern[0])] + substitute(pattern[1:], pat_dic)

def pat_match_with_seg(pattern, saying):
    "Revise the previous pat_match in order to match ?*x variable with a segment of texts."
    
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):    # when we have a ?X variable
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):   # when we have a ?*X variable
        if segment_match(pattern, saying) != fail:
            match, index = segment_match(pattern, saying)
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
        else: 
            return segment_match(pattern, saying) # when the segment_match returns fail
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    

def segment_match(pattern, saying):
    "Find the group of words in saying that matches with the ?*x variable in a given pattern."
    
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')  # change the ?* prefix into a single ?
    
    if not rest: return (seg_pat, saying), len(saying) 
    
    for i, token in enumerate(saying):
        if rest[0] == token: 
            return (seg_pat, saying[:i]), i    # note i = len(saying) :)
    
    return fail # to prevent ?*X matches the whole of saying


def get_response(saying, pat_dic):
    if not saying or not pat_dic: return []
    
    response_list = []
    
    for key, value in pat_dic.items():
        got_patterns = pat_match_with_seg(key.split(), saying.split())
        if got_patterns != fail:
            pat_response = pat_dic[key]
            for r in pat_response:
                response_list.append(' '.join(substitute(r.split(), pat_to_dic(got_patterns))))
        else: continue
    
    if response_list: 
        return random.choice(response_list)
    else: 
        return "Sorry, I don't know how to answer."


In [142]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
        '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

#### 问题 3 
多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

#### 问题4

1. 这样的程序有什么优缺点？你有什么可以改进的方法吗？
2. 什么是数据驱动？数据驱动在这个程序里如何体现？
3. 数据驱动与AI的关系是什么？